In [1]:
from opendbc.car import structs
from opendbc.car.honda.values import CAR

TEST_PLATFORMS = set(CAR)

print(f"Test set contains {len(TEST_PLATFORMS)} qualifying platforms:")
for platform in TEST_PLATFORMS:
  print(f"{platform}")

Test set contains 23 qualifying platforms:
HONDA_CRV
HONDA_ODYSSEY_CHN
ACURA_ILX
HONDA_E
HONDA_CIVIC_BOSCH
ACURA_RDX_3G
HONDA_HRV
HONDA_INSIGHT
HONDA_ODYSSEY
ACURA_RDX
HONDA_FREED
HONDA_CIVIC
HONDA_FIT
HONDA_CIVIC_2022
HONDA_CRV_5G
HONDA_ACCORD
HONDA_CRV_EU
HONDA_RIDGELINE
HONDA_CRV_HYBRID
HONDA_CIVIC_BOSCH_DIESEL
HONDA_PILOT
HONDA_PILOT_4G
HONDA_HRV_3G


In [2]:
import random

from openpilot.tools.lib.logreader import LogReader
from openpilot.tools.lib.comma_car_segments import get_comma_car_segments_database

database = get_comma_car_segments_database()
TEST_SEGMENTS = []

MAX_SEGS_PER_PLATFORM = 5  # TODO: Increase this to search more segments

print("Collecting segments from commaCarSegments dataset:\n")
for platform in TEST_PLATFORMS:
  # assert(platform in database)
  if platform not in database:
    print(f"Skipping platform: {platform}, no data available")
    continue
  
  all_segments = database[platform]

  NUM_SEGMENTS = min(len(all_segments), MAX_SEGS_PER_PLATFORM)

  print(f"Got {len(all_segments)} segments for platform {platform}, sampling {NUM_SEGMENTS} segments")

  segment_subset = random.sample(all_segments, NUM_SEGMENTS)
  for segment in segment_subset:
    TEST_SEGMENTS.append([platform, segment])

print(f"\nSampled {len(TEST_SEGMENTS)} segments from commaCarSegments")

# For new car port development, not yet upstream
EXTRA_SEGMENTS = [
  ["ACURA_MDX_4G", "f9c43864cf057d05|2024-01-15--23-01-20/0"],  # CAN-FD
  ["ACURA_INTEGRA", "3f8ae015ce70365f/00000003--a22590d0e4/0"],
  ["HONDA_CITY", "56b2cf1dacdcd033/00000048--3267801001/0"],
  # ["HONDA_ODYSSEY_5G_MMR", "558ac87ada6a88cd/00000036--7d59e2af08/0"],  # no rlogs
  ["ACURA_MDX_4G_MMR", "ad9840558640c31d/0000001a--d6cd4871c2/0"],  # CAN-FD
  ["ACURA_RDX_3G_MMR", "0416c2843c0bfc91/0000001d--59c7744d87/0"],
  ["HONDA_NBOX", "414af83891dbf72c/00000006--51fa6d99cd/0"],
]

TEST_SEGMENTS.extend(EXTRA_SEGMENTS)

print(f"Added {len(EXTRA_SEGMENTS)} manual segments")



Got 156 segments for platform HONDA_CRV, sampling 5 segments
Skipping platform: HONDA_ODYSSEY_CHN, no data available
Got 4 segments for platform ACURA_ILX, sampling 4 segments
Got 2 segments for platform HONDA_E, sampling 2 segments
Got 12 segments for platform HONDA_CIVIC_BOSCH, sampling 5 segments
Got 747 segments for platform ACURA_RDX_3G, sampling 5 segments
Got 150 segments for platform HONDA_HRV, sampling 5 segments
Got 661 segments for platform HONDA_INSIGHT, sampling 5 segments
Got 19 segments for platform HONDA_ODYSSEY, sampling 5 segments
Got 147 segments for platform ACURA_RDX, sampling 5 segments
Got 50 segments for platform HONDA_FREED, sampling 5 segments
Got 1483 segments for platform HONDA_CIVIC, sampling 5 segments
Got 282 segments for platform HONDA_FIT, sampling 5 segments
Got 1341 segments for platform HONDA_CIVIC_2022, sampling 5 segments
Got 3836 segments for platform HONDA_CRV_5G, sampling 5 segments
Got 16 segments for platform HONDA_ACCORD, sampling 5 segments

In [15]:
import copy
from math import isclose
import matplotlib.pyplot as plt
import numpy as np

from opendbc.can.parser import CANParser
from opendbc.car.honda.values import DBC
from opendbc.car.honda.hondacan import CanBus

from openpilot.selfdrive.pandad import can_capnp_to_list
from openpilot.tools.lib.logreader import LogReader

door_address = 1029
position_addresses = [392, 401, 419]
mt_address = 477
scm_addresses = [660, 806, 422, 662]
all_addresses = [*position_addresses, *scm_addresses, mt_address, door_address]
buses = [0, 1]

car_results = {}
for segment in TEST_SEGMENTS:
  car_results[segment[0]] = {}
  for address in all_addresses:
    car_results[segment[0]][address] = 0

for segment in TEST_SEGMENTS:
  print(f"Analyzing segment {segment[1]:<44} for {segment[0]}")
  lr = LogReader(segment[1])
  CP = lr.first("carParams")
  if CP is None:
    continue


  segment_counts = {}
  gear_activity = {}
  for address in all_addresses:
    segment_counts[address] = 0
    gear_activity[address] = False
    
  can_msgs = [msg for msg in lr if msg.which() == "can"]
  for can_msg in can_msgs:
    for packet in can_msg.can:
      for address in all_addresses:
        if packet.src in buses and packet.address == address:
          if address == 392:
            assert len(packet.dat) == 6
            segment_counts[392] += 1
            if packet.dat[0] & 0x7F != 0:
              gear_activity[address] = True
          if address == 401:
            assert len(packet.dat) == 8
            segment_counts[401] += 1
            if packet.dat[3] & 0xF != 0:
              gear_activity[address] = True
          if address == 419:
            assert len(packet.dat) == 8
            segment_counts[419] += 1
            if packet.dat[3] & 0x7F != 0:
              gear_activity[address] = True
          if address == 422:
            # SCM_BUTTONS A
            assert len(packet.dat) == 8
            segment_counts[422] += 1
          if address == 477:
            # Only used for MT, but often present on non-MT cars, not as easy to detect gear activity
            assert len(packet.dat) == 8
            segment_counts[477] += 1
          if address == 660:
            # SCM_FEEDBACK A
            assert len(packet.dat) == 8
            segment_counts[660] += 1
          if address == 662:
            # SCM_BUTTONS B
            packetlen = len(packet.dat)
            if packetlen != 4:
              print(f"622 length {packetlen}")
            segment_counts[662] += 1
          if address == 806:
            # SCM_FEEDBACK B
            assert len(packet.dat) == 8
            segment_counts[806] += 1
          if address == 1029:
            assert len(packet.dat) == 8
            segment_counts[1029] += 1

  # Make sure gear position activity was found, or the MT message is present
  #print(segment_counts)
    
  gear_position_found = False
  for address in position_addresses:
    if segment_counts[address] > 0:
      frequency = round(segment_counts[address] / 60)
      if gear_activity[address]:
        #print(f"Found shifter position: GEARBOX_{address} at {frequency}Hz")
        #assert(address == 392 and isclose(frequency, 100, abs_tol=20) or \
        #       address == 401 and isclose(frequency, 100, abs_tol=20) or \
        #       address == 419 and isclose(frequency, 50, abs_tol=10))               
        car_results[segment[0]][address] += 1
        gear_position_found = True

  if not gear_position_found:
    address = mt_address
    frequency = round(segment_counts[address] / 60)
    #print(f"Found manual trans car: GEARBOX_477 at {frequency}Hz")
    #assert isclose(frequency, 50, abs_tol=5)
    car_results[segment[0]][address] += 1

  if segment_counts[door_address] > 0:
    frequency = round(segment_counts[door_address] / 60)
    #print(f"Found door message: DOORS_OPEN at {frequency}Hz")
    #assert isclose(frequency, 3, abs_tol=1)
    car_results[segment[0]][door_address] += 1

  for scm_address in scm_addresses:
    if segment_counts[scm_address] > 0:
      frequency = round(segment_counts[scm_address] / 60)
      print(f"Found SCM message {scm_address=} at {frequency}Hz")
      car_results[segment[0]][scm_address] += 1

print(f"Analysis finished")


Analyzing segment aa2185702e887680/00000052--e5da04584a/17/s   for HONDA_CRV
Found SCM message scm_address=660 at 25Hz
Found SCM message scm_address=422 at 50Hz
Analyzing segment 3f140ce4ed67c267/00000195--7abfcd299a/61/s   for HONDA_CRV
Found SCM message scm_address=660 at 25Hz
Found SCM message scm_address=422 at 50Hz
Analyzing segment aa2185702e887680/00000077--bfe079b3a8/53/s   for HONDA_CRV
Found SCM message scm_address=660 at 25Hz
Found SCM message scm_address=422 at 50Hz
Analyzing segment a4b5df8d28022ac1/00000000--7097b5b521/15/s   for HONDA_CRV
Found SCM message scm_address=660 at 25Hz
Found SCM message scm_address=422 at 50Hz
Analyzing segment 3f140ce4ed67c267/000001b7--a90a4b676b/32/s   for HONDA_CRV
Found SCM message scm_address=660 at 25Hz
Found SCM message scm_address=422 at 50Hz
Analyzing segment fba0ac10cc1b90ac/00000040--b373054b96/1/s    for ACURA_ILX
Found SCM message scm_address=660 at 25Hz
Found SCM message scm_address=422 at 50Hz
Analyzing segment fba0ac10cc1b90ac

In [12]:
print(f"Gear messages by platform      0x188 0x191 0x1A3 0x1DD 0x405   0x294 0x1A6 0x326 0x296")
print(f"------------------------------ ----- ----- ----- ----- -----   ----- ----- ----- -----")
for platform in sorted(car_results):
  print(f"{platform:30} {car_results[platform][392]:5} {car_results[platform][401]:5} {car_results[platform][419]:5} {car_results[platform][477]:5} {car_results[platform][1029]:5}" + \
                      f"{car_results[platform][660]:5} {car_results[platform][422]:5} {car_results[platform][806]:5} {car_results[platform][662]:5}")


Gear messages by platform      0x188 0x191 0x1A3 0x1DD 0x405   0x294 0x1A6 0x326 0x296
------------------------------ ----- ----- ----- ----- -----   ----- ----- ----- -----
ACURA_ILX                          0     0     4     0     4    4     4     0     0
ACURA_INTEGRA                      0     1     0     0     0    0     0     1     1
ACURA_MDX_4G                       0     0     1     0     0    0     0     1     1
ACURA_MDX_4G_MMR                   0     0     1     0     0    0     0     1     1
ACURA_RDX                          5     0     0     0     5    5     5     0     0
ACURA_RDX_3G                       0     0     5     0     0    0     0     5     5
ACURA_RDX_3G_MMR                   0     0     0     0     0    0     0     0     0
HONDA_ACCORD                       0     5     0     0     0    0     0     5     5
HONDA_CITY                         0     1     0     0     0    0     0     1     1
HONDA_CIVIC                        0     5     0     0     5    0     